In [3]:
import tensorflow as tf

In [ ]:
# Fungsi masked loss
def masked_mse_loss(y_true, y_pred):
    mask = tf.not_equal(y_true, 0)  # Masker untuk nilai yang tidak nol (keypoint yang tersedia)
    mask = tf.cast(mask, tf.float32)  # Konversi ke float
    loss = tf.square(y_true - y_pred) * mask  # Hitung MSE hanya pada keypoint yang tersedia
    loss = tf.reduce_sum(loss) / tf.reduce_sum(mask)  # Rata-rata loss berdasarkan jumlah keypoint yang tersedia
    return loss

model_path = '/content/drive/MyDrive/Colab Notebooks/model/facial keypoints detection model/facial_keypoints_detection_model.keras'
best_model = tf.keras.models.load_model(
    model_path,
    custom_objects={'masked_mse': masked_mse_loss}
)

In [ ]:
best_model.summary()

In [1]:
# Fungsi untuk membuat blok residual dengan konvolusi strided
def residual_block(x, filters):
    residual = x
    x = tf.keras.layers.Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.Conv2D(filters, (3, 3), activation='relu', padding='same', strides=2)(x)
    residual = tf.keras.layers.Conv2D(filters, (1, 1), padding='same', strides=2)(residual)
    x = tf.keras.layers.add([x, residual])
    return x

# Arsitektur ML-ResNet
def create_ml_resnet(input_shape):
    inputs = tf.keras.layers.Input(shape=input_shape)

    # Lapisan pertama
    x = tf.keras.layers.Conv2D(32, (11, 11), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)

    # Blok residual pertama
    x = residual_block(x, 64)

    # Blok residual kedua
    x = residual_block(x, 128)

    # Lapisan akhir
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1000, activation='relu')(x)
    x = tf.keras.layers.Dense(30)(x)  # 15 keypoints dengan masing-masing x dan y koordinat

    model = tf.keras.models.Model(inputs, x)
    return model

load_model=create_ml_resnet((96,96,1))
load_model.load_weights('/content/drive/MyDrive/Colab Notebooks/model/facial keypoints detection model/facial_keypoints_detection_model.keras')

In [ ]:
face_images_test_tf_dataset_loaded = tf.data.Dataset.load("/content/drive/MyDrive/Colab Notebooks/dataset/facial_keypoints_test_dataset.tfrecord", compression="GZIP")

In [ ]:
print(f'info data: {face_images_test_tf_dataset_loaded}')
print(f'number of data: {len(face_images_test_tf_dataset_loaded)}')

In [ ]:
for img in face_images_test_tf_dataset_loaded.skip(1).take(1):
  print(f'''
    max-intensity: {tf.reduce_max(img)}
    min-intensity: {tf.reduce_min(img)}
    image-shape: {img.shape}
    image-type: {img.dtype}
  ''')

In [ ]:
def processed_test_image(image):
  image = tf.cast(image, tf.float32)
  image = image / 255.0
  return image

In [ ]:
face_images_test_tf_dataset_fixed_processed = face_images_test_tf_dataset_loaded.map(processed_test_image, num_parallel_calls=tf.data.AUTOTUNE)
facial_keypoints_concatenated_test_dataset_batched = face_images_test_tf_dataset_fixed_processed.batch(64)
facial_keypoints_concatenated_test_dataset_cached = facial_keypoints_concatenated_test_dataset_batched.cache()
facial_keypoints_concatenated_test_dataset_prefetched = facial_keypoints_concatenated_test_dataset_cached.prefetch(tf.data.AUTOTUNE)

In [ ]:
for img in face_images_test_tf_dataset_fixed_processed.skip(1).take(1):
  print(f'''
    max-intensity: {tf.reduce_max(img)}
    min-intensity: {tf.reduce_min(img)}
    image-shape: {img.shape}
    image-type: {img.dtype}
  ''')

In [ ]:
idx = np.random.randint(0, len(facial_keypoints_concatenated_test_dataset_prefetched))
plt.figure(figsize=(10,5))
for image in facial_keypoints_concatenated_test_dataset_prefetched.skip(idx).take(1):
  image = tf.expand_dims(image[0], axis=0)
  pred_keypoints = best_model.predict(image)
  pred_keypoints = tf.squeeze(pred_keypoints)

  plt.imshow(image[0], cmap='gray')
  for i in range(0, len(pred_keypoints), 2):
    x = pred_keypoints[i].numpy()
    y = pred_keypoints[i+1].numpy()
    plt.scatter(x, y, s=200, marker='.', c='blue')
  plt.title('prediction')

plt.show()